In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Configuration
NUM_ASSETS = 5000
START_DATE = datetime(2018, 1, 1)
END_DATE = datetime(2024, 10, 31)

# Reference data
MANUFACTURERS = ['Dell', 'HP', 'Lenovo', 'Apple', 'Microsoft']
ASSET_TYPES = ['Laptop', 'Desktop', 'Monitor', 'Server', 'Tablet']
REGIONS = ['North America', 'Europe', 'Asia Pacific', 'Latin America']
DEPARTMENTS = ['IT', 'Finance', 'HR', 'Sales', 'Marketing', 'Operations', 'Engineering']
OS_VERSIONS = ['Windows 10', 'Windows 11', 'macOS Ventura', 'macOS Sonoma', 'Ubuntu 20.04', 'Ubuntu 22.04']
STATUS = ['Active', 'In Storage', 'In Repair', 'Retired', 'Disposed']
ENCRYPTION_STATUS = ['Encrypted', 'Not Encrypted', 'Unknown']

# Model configurations with realistic pricing and lifecycles
MODELS = {
    'Dell': ['Latitude 5420', 'Latitude 7420', 'OptiPlex 7090', 'PowerEdge R740'],
    'HP': ['EliteBook 840', 'ProBook 450', 'EliteDesk 800', 'ProLiant DL380'],
    'Lenovo': ['ThinkPad X1 Carbon', 'ThinkPad T14', 'ThinkCentre M90', 'ThinkSystem SR650'],
    'Apple': ['MacBook Pro 14"', 'MacBook Air M1', 'MacBook Pro 16"', 'Mac Mini M1'],
    'Microsoft': ['Surface Laptop 4', 'Surface Pro 8', 'Surface Book 3', 'Surface Go 3']
}

def generate_serial_number(manufacturer, index):
    """Generate realistic serial numbers"""
    prefix = manufacturer[:3].upper()
    return f"{prefix}{random.randint(100000, 999999)}{index:04d}"

def generate_purchase_date():
    """Generate purchase date within range"""
    days_range = (END_DATE - START_DATE).days
    random_days = random.randint(0, days_range)
    return START_DATE + timedelta(days=random_days)

def generate_asset_cost(asset_type, manufacturer):
    """Generate realistic asset costs"""
    base_costs = {
        'Laptop': (800, 2500),
        'Desktop': (600, 1500),
        'Monitor': (200, 800),
        'Server': (5000, 15000),
        'Tablet': (400, 1200)
    }

    base_min, base_max = base_costs.get(asset_type, (500, 2000))

    # Premium manufacturers cost more
    multiplier = 1.3 if manufacturer in ['Apple', 'Microsoft'] else 1.0

    return round(random.uniform(base_min, base_max) * multiplier, 2)

def generate_last_login_days(status, purchase_date):
    """Generate days since last login based on status"""
    days_owned = (END_DATE - purchase_date).days

    if status == 'Active':
        return random.randint(0, 7)
    elif status == 'In Storage':
        return random.randint(30, 365)
    elif status == 'In Repair':
        return random.randint(7, 30)
    else:  # Retired or Disposed
        lower_bound_for_retired = 180
        if days_owned < lower_bound_for_retired:
            # If the asset hasn't been owned for 180 days, it cannot have been last logged in 180+ days ago.
            # In this case, the last login must be within the asset's ownership period.
            # We'll set a floor of 30 days for 'inactive' status, but not exceeding days_owned.
            return random.randint(min(30, days_owned), days_owned) if days_owned > 0 else 0
        else:
            return random.randint(lower_bound_for_retired, days_owned)

def generate_cpu_usage(status):
    """Generate average CPU usage percentage"""
    if status == 'Active':
        return round(random.uniform(20, 85), 2)
    elif status in ['In Storage', 'Retired', 'Disposed']:
        return 0.0
    else:  # In Repair
        return round(random.uniform(0, 30), 2)

def generate_disk_usage(status):
    """Generate disk usage percentage"""
    if status in ['Retired', 'Disposed']:
        return 0.0
    else:
        return round(random.uniform(30, 95), 2)

def determine_failure_risk(age_years, manufacturer, asset_type, cpu_usage):
    """Determine if asset has failed or will fail soon"""
    # Base failure probability increases with age
    base_prob = min(age_years * 0.08, 0.6)

    # Manufacturer reliability factor
    reliability = {'Dell': 0.95, 'HP': 0.92, 'Lenovo': 0.94, 'Apple': 0.97, 'Microsoft': 0.93}
    base_prob *= (1 / reliability.get(manufacturer, 0.93))

    # Asset type factor
    if asset_type == 'Server':
        base_prob *= 1.3
    elif asset_type == 'Laptop':
        base_prob *= 1.1

    # High usage increases failure risk
    if cpu_usage > 75:
        base_prob *= 1.2

    return random.random() < base_prob

# Generate main asset dataset
print("Generating asset data...")
assets = []

for i in range(NUM_ASSETS):
    manufacturer = random.choice(MANUFACTURERS)
    asset_type = random.choice(ASSET_TYPES)
    model = random.choice(MODELS[manufacturer])
    purchase_date = generate_purchase_date()
    age_years = (END_DATE - purchase_date).days / 365.25

    # Status influenced by age
    if age_years > 5:
        status = random.choices(STATUS, weights=[0.3, 0.1, 0.1, 0.3, 0.2])[0]
    elif age_years > 3:
        status = random.choices(STATUS, weights=[0.6, 0.15, 0.1, 0.1, 0.05])[0]
    else:
        status = random.choices(STATUS, weights=[0.85, 0.05, 0.05, 0.03, 0.02])[0]

    last_login_days = generate_last_login_days(status, purchase_date)
    cpu_usage = generate_cpu_usage(status)
    disk_usage = generate_disk_usage(status)
    has_failed = determine_failure_risk(age_years, manufacturer, asset_type, cpu_usage)

    # Introduce data quality issues intentionally (10% of records)
    encryption_status = ENCRYPTION_STATUS[0] if random.random() > 0.15 else random.choice(ENCRYPTION_STATUS)
    assigned_user = f"user{random.randint(1000, 9999)}" if random.random() > 0.05 else None
    warranty_end = purchase_date + timedelta(days=random.choice([365, 1095, 1825])) if random.random() > 0.08 else None

    asset = {
        'Asset_ID': f'AST{i+1:06d}',
        'Serial_Number': generate_serial_number(manufacturer, i),
        'Manufacturer': manufacturer,
        'Model': model,
        'Asset_Type': asset_type,
        'Purchase_Date': purchase_date.strftime('%Y-%m-%d'),
        'Purchase_Cost': generate_asset_cost(asset_type, manufacturer),
        'Warranty_End_Date': warranty_end.strftime('%Y-%m-%d') if warranty_end else None,
        'Region': random.choice(REGIONS),
        'Department': random.choice(DEPARTMENTS),
        'Assigned_User': assigned_user,
        'Status': status,
        'OS_Version': random.choice(OS_VERSIONS) if asset_type in ['Laptop', 'Desktop'] else None,
        'Encryption_Status': encryption_status if asset_type in ['Laptop', 'Desktop', 'Tablet'] else None,
        'Last_Login_Days_Ago': last_login_days,
        'Avg_CPU_Usage_Percent': cpu_usage,
        'Disk_Usage_Percent': disk_usage,
        'Has_Failed': 'Yes' if has_failed else 'No',
        'Age_Years': round(age_years, 2)
    }

    assets.append(asset)

df_assets = pd.DataFrame(assets)

# Generate ServiceNow discovery data (with some discrepancies for data quality analysis)
print("Generating ServiceNow discovery data...")
servicenow_data = []

for _, asset in df_assets.iterrows():
    if random.random() > 0.15:  # 85% of assets appear in ServiceNow
        # Introduce data quality issues
        discovered_user = asset['Assigned_User'] if random.random() > 0.12 else f"user{random.randint(1000, 9999)}"
        discovered_encryption = asset['Encryption_Status'] if random.random() > 0.10 else random.choice(ENCRYPTION_STATUS)
        last_discovered = END_DATE - timedelta(days=random.randint(0, 90))

        sn_record = {
            'CI_ID': f"CI{random.randint(100000, 999999)}",
            'Serial_Number': asset['Serial_Number'],
            'Discovered_User': discovered_user,
            'Discovered_Encryption': discovered_encryption,
            'Last_Discovery_Date': last_discovered.strftime('%Y-%m-%d'),
            'Discovery_Source': random.choice(['SCCM', 'JAMF', 'Intune'])
        }
        servicenow_data.append(sn_record)

df_servicenow = pd.DataFrame(servicenow_data)

# Generate failure/incident history
print("Generating incident history...")
incidents = []
incident_id = 1

for _, asset in df_assets[df_assets['Has_Failed'] == 'Yes'].iterrows():
    num_incidents = random.randint(1, 3)
    purchase_date = datetime.strptime(asset['Purchase_Date'], '%Y-%m-%d')

    days_owned = (END_DATE - purchase_date).days

    # Skip incident generation if the asset has been owned for 0 or negative days
    # or if the asset has been owned for less than 1 day (cannot have an incident 'after purchase')
    if days_owned <= 0:
        continue

    # Ensure the lower bound for days_after_purchase does not exceed the total days the asset has been owned
    # Incidents are generally assumed to happen at least 180 days after purchase, but not before the asset exists.
    lower_bound_for_incident_days = max(1, min(180, days_owned))

    for _ in range(num_incidents):
        days_after_purchase = random.randint(lower_bound_for_incident_days, days_owned)
        incident_date = purchase_date + timedelta(days=days_after_purchase)

        incident = {
            'Incident_ID': f'INC{incident_id:06d}',
            'Asset_ID': asset['Asset_ID'],
            'Serial_Number': asset['Serial_Number'],
            'Incident_Date': incident_date.strftime('%Y-%m-%d'),
            'Issue_Type': random.choice(['Hardware Failure', 'Performance Issue', 'Software Issue', 'Network Issue']),
            'Resolution_Days': random.randint(1, 15),
            'Repair_Cost': round(random.uniform(50, 500), 2)
        }
        incidents.append(incident)
        incident_id += 1

df_incidents = pd.DataFrame(incidents)

# Save datasets
print("Saving datasets...")
df_assets.to_csv('it_assets_master.csv', index=False)
df_servicenow.to_csv('servicenow_discovery.csv', index=False)
df_incidents.to_csv('incident_history.csv', index=False)

# Generate data dictionary
data_dict = """
# IT Asset Management Dataset - Data Dictionary

## File 1: it_assets_master.csv (Master Asset Database)
- Asset_ID: Unique identifier for each asset
- Serial_Number: Manufacturer serial number
- Manufacturer: Asset manufacturer (Dell, HP, Lenovo, Apple, Microsoft)
- Model: Specific model name
- Asset_Type: Category (Laptop, Desktop, Monitor, Server, Tablet)
- Purchase_Date: Date asset was purchased
- Purchase_Cost: Original purchase cost in USD
- Warranty_End_Date: Date warranty expires (may be null for data quality issues)
- Region: Geographic region where asset is deployed
- Department: Department to which asset is assigned
- Assigned_User: User ID of assigned user (may be null)
- Status: Current asset status (Active, In Storage, In Repair, Retired, Disposed)
- OS_Version: Operating system version (for computers only)
- Encryption_Status: Disk encryption status (Encrypted, Not Encrypted, Unknown)
- Last_Login_Days_Ago: Days since last user login
- Avg_CPU_Usage_Percent: Average CPU utilization (last 30 days)
- Disk_Usage_Percent: Percentage of disk space used
- Has_Failed: Whether asset has experienced failure (Yes/No)
- Age_Years: Asset age in years

## File 2: servicenow_discovery.csv (Discovery Tool Data)
- CI_ID: Configuration Item ID in ServiceNow
- Serial_Number: Serial number (should match master data)
- Discovered_User: User discovered by automated tools
- Discovered_Encryption: Encryption status from discovery
- Last_Discovery_Date: Last successful discovery scan date
- Discovery_Source: Discovery tool used (SCCM, JAMF, Intune)

## File 3: incident_history.csv (Support Ticket History)
- Incident_ID: Unique incident identifier
- Asset_ID: Associated asset ID
- Serial_Number: Asset serial number
- Incident_Date: Date incident was logged
- Issue_Type: Category of issue reported
- Resolution_Days: Days taken to resolve
- Repair_Cost: Cost of repair in USD

## Data Quality Issues (Intentional for Analysis)
- ~15% of assets missing in ServiceNow discovery
- ~12% user assignment discrepancies between systems
- ~10% encryption status discrepancies
- ~8% missing warranty end dates
- ~5% missing assigned users
"""

with open('DATA_DICTIONARY.md', 'w') as f:
    f.write(data_dict)

# Print summary statistics
print("\n" + "="*60)
print("DATASET GENERATION COMPLETE")
print("="*60)
print(f"\nTotal Assets Generated: {len(df_assets):,}")
print(f"ServiceNow Records: {len(df_servicenow):,}")
print(f"Incident Records: {len(df_incidents):,}")
print(f"\nAsset Status Distribution:")
print(df_assets['Status'].value_counts())
print(f"\nAsset Type Distribution:")
print(df_assets['Asset_Type'].value_counts())
print(f"\nAssets with Failures: {len(df_assets[df_assets['Has_Failed'] == 'Yes']):,}")
print("\nFiles created:")
print("  1. it_assets_master.csv")
print("  2. servicenow_discovery.csv")
print("  3. incident_history.csv")
print("  4. DATA_DICTIONARY.md")
print("\n" + "="*60)


Generating asset data...
Generating ServiceNow discovery data...
Generating incident history...
Saving datasets...

DATASET GENERATION COMPLETE

Total Assets Generated: 5,000
ServiceNow Records: 4,280
Incident Records: 3,114

Asset Status Distribution:
Status
Active        3138
Retired        649
In Storage     463
In Repair      396
Disposed       354
Name: count, dtype: int64

Asset Type Distribution:
Asset_Type
Laptop     1074
Desktop     998
Monitor     981
Server      980
Tablet      967
Name: count, dtype: int64

Assets with Failures: 1,555

Files created:
  1. it_assets_master.csv
  2. servicenow_discovery.csv
  3. incident_history.csv
  4. DATA_DICTIONARY.md

